In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [2]:
events_data = pd.read_csv('event_data_train.csv')
submission_data = pd.read_csv('submissions_data_train.csv')

In [3]:
submission_data.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [4]:
submission_data.shape

(509104, 4)

In [5]:
# submission_data.loc[submission_data.submission_status == 'wrong'].groupby('step_id')\
# .agg({'submission_status' : 'count'}).sort_values(by = 'submission_status',ascending = False)


In [6]:
submissions_data =  submission_data

In [8]:
submissions_data.sort_values(['user_id', 'timestamp'], ascending=False)\
.drop_duplicates(['user_id'])\
        .query("submission_status == 'wrong'").groupby('step_id')\
        .count().sort_values('submission_status').tail(1)

,step_id,timestamp,submission_status,user_id
507609,32812,1520978070,correct,26798
507608,32812,1520978063,wrong,26798
507610,32929,1520977976,correct,26798
506152,32812,1508425828,correct,26797
506153,32929,1508425367,correct,26797
...,...,...,...,...
114951,32219,1434359290,wrong,3
114956,32812,1434358574,correct,3
114957,32929,1434358533,correct,3
506853,32812,1514383485,correct,2


In [10]:
submissions_data_by_step = submissions_data \
    .pivot_table(values='timestamp', index=['user_id', 'step_id'], aggfunc = 'max', 
                 columns='submission_status', fill_value=0) \
    .reset_index()

submissions_data_by_step.head()

submission_status,user_id,step_id,correct,wrong
0,2,32812,1514383485,0
1,2,32929,1514383420,0
2,3,31971,1441259266,0
3,3,31972,1441258743,0
4,3,31976,1441258564,0


In [ ]:
submissions_data_by_step['is_solved'] = submissions_data_by_step.correct > 0

users_max_time = submissions_data.groupby('user_id', as_index=False) \
    .agg({'timestamp': 'max'}) \
    .rename(columns={'timestamp': 'last_timestamp'})


not_solved_steps = submissions_data_by_step[submissions_data_by_step.is_solved == False]

not_solved_steps = not_solved_steps.merge(users_max_time, on='user_id')

not_solved_steps['is_last_action'] = not_solved_steps.wrong == not_solved_steps.last_timestamp

not_solved_steps_last_action = not_solved_steps[not_solved_steps.is_last_action == True]

not_solved_steps_last_action.groupby('step_id', as_index=False) \
    .agg({'user_id': 'count'}).sort_values('user_id', ascending=False).head()
